In [1]:
import requests
import pandas as pd
import numpy as np
import uuid

pd.options.mode.chained_assignment = None

In [7]:
df_raw = pd.read_csv('blocks.csv')
df_raw.drop(columns=['transactions'], inplace=True)
df_raw.sort_values(by='block_number', ascending=True, inplace=True)
df_raw.reset_index(drop=True, inplace=True)
df_raw.to_csv('blocks.csv', index=False)

In [ ]:
url = f'https://blocks.flashbots.net/v1/blocks?limit=5000'

response = requests.get(url)

raw = response.json()
raw = raw['blocks']

df_raw = pd.DataFrame.from_dict(raw)
df_raw.sort_values(by='block_number', ascending=True, inplace=True)
df_raw.reset_index(drop=True, inplace=True)


def create_transactions(raw):
    txn = []
    for i in range(0, len(raw)):
        txn.append(raw[i]['transactions'])
    
    txn = [item for sublist in txn for item in sublist]
    return txn

txn = create_transactions(raw)
df_txn = pd.DataFrame.from_dict(txn)

df_txn.to_csv('txn.csv', mode='a', index=False)

df_raw.drop(columns=['transactions'])
df_raw.to_csv('blocks.csv', mode='a', index=False)





In [ ]:
temp = pd.read_csv('txn.csv')
temp.drop_duplicates(keep='first', inplace=True)
temp.sort_values(by='block_number', ascending=True, inplace=True)
temp.reset_index(drop=True, inplace=True)


In [ ]:
def create_bundles(df_txn):
    blocks = np.unique(df_txn['block_number'])
    bdl = []
    for i in range(0, len(blocks)):
        bundles = np.unique(df_txn[df_txn['block_number'] == blocks[i]]['bundle_index'])
        for j in range(0, len(bundles)):
            temp = df_txn[(df_txn['block_number'] == blocks[i]) & (df_txn['bundle_index'] == bundles[j])]
            temp['id'] = uuid.uuid4()
            temp.reset_index(drop=True, inplace=True)
            bdl.append(temp.to_dict())

    return bdl

bdl = create_bundles(df_txn)
bdl_df = pd.DataFrame.from_dict(bdl)